<a href="https://colab.research.google.com/github/KorStats/classification-project/blob/main/%EB%B3%B4%EA%B3%A0%EC%84%9C/C%EB%8D%B0%EC%9D%B4%ED%84%B0%EC%A6%9D%EB%B6%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## 구글드라이브에서 파일 읽어오기

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
!ln -s /content/gdrive/My\ Drive/ /mydrive
!ls /mydrive

Mounted at /content/gdrive
 산업분류자동화    '답안 작성용 파일.csv'   df_J_inc.csv  'My Drive'
'Colab Notebooks'  '# df_G_inc.csv'	    df_K_inc.csv   Q_inc_model.pt
 C_ori_model.pt     df_G_inc.csv	    df_P_inc.csv   spell_check


In [ ]:
import pandas as pd
df=pd.read_csv('/content/gdrive/MyDrive/spell_check/C_spellcheck.csv')

In [ ]:
df=df[(df['digit_1'] =='C')] #조건에 맞는 데이터 출력

In [ ]:
df=df.reset_index(drop=True)
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000006,C,29,291,철 절삭. 용접 카프라 배관자재
1,id_0000008,C,10,107,쌀을 가지고 가공하여 떡 제조
2,id_0000011,C,10,102,"멸치 입고, 가공"
3,id_0000021,C,10,107,쌀을 가지고 가루로 분쇄하여 떡을 쪄서 백설기 꿀떡 절편 판매
4,id_0000027,C,22,221,"고무 절단가공 개스킷, 다이아 프레임"
...,...,...,...,...,...
105187,id_0999971,C,28,281,"트랜스, 다이오드, 칩 조립 전원공급장치(파워 서플라이)"
105188,id_0999976,C,25,259,기타 임가공
105189,id_0999986,C,10,106,벼를 가지고 벼를 도정하여 소비자에게 판매
105190,id_0999994,C,32,320,"카본 열선, 퀼팅 구조체 조립. 배열. 봉제하여 의료용 매트리스"


In [ ]:
df['digit_3'].value_counts()

259    12876
107     9214
292     7163
222     4935
291     4924
       ...  
304       22
301       20
191       20
266        8
120        3
Name: digit_3, Length: 85, dtype: int64

In [ ]:
# 데이터 증분 코드
import random
import pickle
import re
def get_only_hangul(line):
	parseText= re.compile('/ ^[ㄱ-ㅎㅏ-ㅣ가-힣]*$/').sub('',line)
	return parseText

def synonym_replacement(words, n):
	new_words = words.copy()
	random_word_list = list(set([word for word in words]))
	random.shuffle(random_word_list)
	num_replaced = 0
	for random_word in random_word_list:
		synonyms = get_synonyms(random_word)
		if len(synonyms) >= 1:
			synonym = random.choice(list(synonyms))
			new_words = [synonym if word == random_word else word for word in new_words]
			num_replaced += 1
		if num_replaced >= n:
			break
	if len(new_words) != 0:
		sentence = ' '.join(new_words)
		new_words = sentence.split(" ")
	else:
		new_words = ""
	return new_words

def get_synonyms(word):
	synomyms = []
	try:
		for syn in wordnet[word]:
			for s in syn:
				synomyms.append(s)
	except:
		pass
	return synomyms

def random_deletion(words, p):
	if len(words) == 1:
		return words

	new_words = []
	for word in words:
		r = random.uniform(0, 1)
		if r > p:
			new_words.append(word)
	if len(new_words) == 0:
		rand_int = random.randint(0, len(words)-1)
		return [words[rand_int]]
	return new_words

########################################################################
# Random swap
# Randomly swap two words in the sentence n times
########################################################################
def random_swap(words, n):
	new_words = words.copy()
	for _ in range(n):
		new_words = swap_word(new_words)
	return new_words

def swap_word(new_words):
	random_idx_1 = random.randint(0, len(new_words)-1)
	random_idx_2 = random_idx_1
	counter = 0
	while random_idx_2 == random_idx_1:
		random_idx_2 = random.randint(0, len(new_words)-1)
		counter += 1
		if counter > 3:
			return new_words
	new_words[random_idx_1], new_words[random_idx_2] = new_words[random_idx_2], new_words[random_idx_1]
	return new_words

########################################################################
# Random insertion
# Randomly insert n words into the sentence
########################################################################
def random_insertion(words, n):
	new_words = words.copy()
	for _ in range(n):
		add_word(new_words)	
	return new_words

def add_word(new_words):
	synonyms = []
	counter = 0
	while len(synonyms) < 1:
		if len(new_words) >= 1:
			random_word = new_words[random.randint(0, len(new_words)-1)]
			synonyms = get_synonyms(random_word)
			counter += 1
		else:
			random_word = ""
		if counter >= 10:
			return
		
	random_synonym = synonyms[0]
	random_idx = random.randint(0, len(new_words)-1)
	new_words.insert(random_idx, random_synonym)

def EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.1, p_rd=0.1, num_aug=9):
	sentence = get_only_hangul(sentence)
	words = sentence.split(' ')
	words = [word for word in words if word is not ""]
	num_words = len(words)
	augmented_sentences = []
	num_new_per_technique = int(num_aug/4) + 1
	n_sr = max(1, int(alpha_sr*num_words))
	n_ri = max(1, int(alpha_ri*num_words))
	n_rs = max(1, int(alpha_rs*num_words))
	# sr
	for _ in range(num_new_per_technique):
		a_words = synonym_replacement(words, n_sr)
		augmented_sentences.append(' '.join(a_words))
	# ri
	for _ in range(num_new_per_technique):
		a_words = random_insertion(words, n_ri)
		augmented_sentences.append(' '.join(a_words))
	# rs
	for _ in range(num_new_per_technique):
		a_words = random_swap(words, n_rs)
		augmented_sentences.append(" ".join(a_words))
	# rd
	for _ in range(num_new_per_technique):
		a_words = random_deletion(words, p_rd)
		augmented_sentences.append(" ".join(a_words))
	augmented_sentences = [get_only_hangul(sentence) for sentence in augmented_sentences]
	random.shuffle(augmented_sentences)
	if num_aug >= 1:
		augmented_sentences = augmented_sentences[:num_aug]
	else:
		keep_prob = num_aug / len(augmented_sentences)
		augmented_sentences = [s for s in augmented_sentences if random.uniform(0, 1) < keep_prob]
	augmented_sentences.append(sentence)
	return augmented_sentences

In [ ]:
df1=df[df['digit_3']==259]
df2=df[df['digit_3']==107]
df3=df[df['digit_3']==292]
df4=df[df['digit_3']==222]
df5=df[df['digit_3']==291]
df6=df[df['digit_3']==141]
df7=df[df['digit_3']==251]
df8=df[df['digit_3']==181]
df9=df[df['digit_3']==339]
df10=df[df['digit_3']==340]
df11=df[df['digit_3']==320]
df12=df[df['digit_3']==132]
df13=df[df['digit_3']==281]
df14=df[df['digit_3']==303]
df15=df[df['digit_3']==262]
df16=df[df['digit_3']==204]
df17=df[df['digit_3']==272]
df18=df[df['digit_3']==271]
df19=df[df['digit_3']==106]
df20=df[df['digit_3']==162]
df21=df[df['digit_3']==284]
df22=df[df['digit_3']==241]
df23=df[df['digit_3']==103]
df24=df[df['digit_3']==172]
df25=df[df['digit_3']==102]
df26=df[df['digit_3']==134]
df27=df[df['digit_3']==239]
df28=df[df['digit_3']==151]
df29=df[df['digit_3']==139]
df30=df[df['digit_3']==221]
df31=df[df['digit_3']==104]
df32=df[df['digit_3']==233]
df33=df[df['digit_3']==311]
df34=df[df['digit_3']==101]
df35=df[df['digit_3']==264]
df36=df[df['digit_3']==232]
df37=df[df['digit_3']==202]
df38=df[df['digit_3']==144]
df39=df[df['digit_3']==331]
df40=df[df['digit_3']==285]
df41=df[df['digit_3']==179]
df42=df[df['digit_3']==242]
df43=df[df['digit_3']==231]
df44=df[df['digit_3']==152]
df45=df[df['digit_3']==131]
df46=df[df['digit_3']==283]
df47=df[df['digit_3']==243]
df48=df[df['digit_3']==261]
df49=df[df['digit_3']==289]
df50=df[df['digit_3']==201]
df51=df[df['digit_3']==161]
df52=df[df['digit_3']==203]
df53=df[df['digit_3']==302]
df54=df[df['digit_3']==143]
df55=df[df['digit_3']==333]
df56=df[df['digit_3']==108]
df57=df[df['digit_3']==263]
df58=df[df['digit_3']==111]
df59=df[df['digit_3']==265]
df60=df[df['digit_3']==133]
df61=df[df['digit_3']==334]
df62=df[df['digit_3']==171]
df63=df[df['digit_3']==273]
df64=df[df['digit_3']==212]
df65=df[df['digit_3']==112]
df66=df[df['digit_3']==213]
df67=df[df['digit_3']==313]
df68=df[df['digit_3']==319]
df69=df[df['digit_3']==312]
df70=df[df['digit_3']==282]
df71=df[df['digit_3']==332]
df72=df[df['digit_3']==192]
df73=df[df['digit_3']==142]
df74=df[df['digit_3']==105]
df75=df[df['digit_3']==205]
df76=df[df['digit_3']==274]
df77=df[df['digit_3']==163]
df78=df[df['digit_3']==211]
df79=df[df['digit_3']==182]
df80=df[df['digit_3']==252]
df81=df[df['digit_3']==304]
df82=df[df['digit_3']==301]
df83=df[df['digit_3']==191]
df84=df[df['digit_3']==266]
df85=df[df['digit_3']==120]


df100=pd.concat([df1, df2, df3, df4, df5, df6])
df101=pd.concat([df7, df8, df9, df10, df11])
df102=pd.concat([df12, df13])
df103=pd.concat([df14, df15, df16, df17, df18, df19, df20, df21])
df104=pd.concat([df22, df23, df24, df25, df26, df27, df28, df29])
df105=pd.concat([df30, df31, df32, df33, df34, df35, df36, df37])
df106=pd.concat([df38, df39, df40, df41, df42, df43])
df107=pd.concat([df44, df45])
df108=pd.concat([df46, df47, df48, df49, df50, df51, df52, df53])
df109=pd.concat([df54, df55, df56, df57])
df110=pd.concat([df58, df59, df60])
df111=pd.concat([df61, df62])
df112=pd.concat([df63])
df113=pd.concat([df64])
df114=pd.concat([df65])
df115=pd.concat([df66, df67, df68, df69, df70, df71, df72, df73])
df116=pd.concat([df74, df75, df76, df77, df78, df79, df80, df81, df82, df83, df84, df85])

df100=df100.reset_index(drop=True) 
df101=df101.reset_index(drop=True) 
df102=df102.reset_index(drop=True) 
df103=df103.reset_index(drop=True) 
df104=df104.reset_index(drop=True) 
df105=df105.reset_index(drop=True) 
df106=df106.reset_index(drop=True) 
df107=df107.reset_index(drop=True) 
df108=df108.reset_index(drop=True) 
df109=df109.reset_index(drop=True) 
df110=df110.reset_index(drop=True) 
df111=df111.reset_index(drop=True) 
df112=df112.reset_index(drop=True) 
df113=df113.reset_index(drop=True) 
df114=df114.reset_index(drop=True) 
df115=df115.reset_index(drop=True) 
df116=df116.reset_index(drop=True) 

## 데이터 증분

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df101['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=1)
  id_list=['p'+str(df101['AI_id'][n]), 'p'+str(df101['AI_id'][n])]
  digit_1_list=[df101['digit_1'][n], df101['digit_1'][n]]
  digit_2_list=[df101['digit_2'][n], df101['digit_2'][n]]
  digit_3_list=[df101['digit_3'][n], df101['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df101_to_insert = pd.DataFrame(data_to_insert)
  df101= df101.append(df101_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df101['digit_3'].value_counts()

251    12468
181    12417
339    10188
340    10140
320     9825
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df102['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=2)
  id_list=['p'+str(df102['AI_id'][n]), 'p'+str(df102['AI_id'][n]), 'p'+str(df102['AI_id'][n])]
  digit_1_list=[df102['digit_1'][n], df102['digit_1'][n], df102['digit_1'][n]]
  digit_2_list=[df102['digit_2'][n], df102['digit_2'][n], df102['digit_2'][n]]
  digit_3_list=[df102['digit_3'][n], df102['digit_3'][n], df102['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df102_to_insert = pd.DataFrame(data_to_insert)
  df102= df102.append(df102_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df102['digit_3'].value_counts()

132    11276
281    11024
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df103['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=3)
  id_list=['p'+str(df103['AI_id'][n]), 'p'+str(df103['AI_id'][n]), 'p'+str(df103['AI_id'][n]), 'p'+str(df103['AI_id'][n])]
  digit_1_list=[df103['digit_1'][n], df103['digit_1'][n], df103['digit_1'][n], df103['digit_1'][n]]
  digit_2_list=[df103['digit_2'][n], df103['digit_2'][n], df103['digit_2'][n], df103['digit_2'][n]]
  digit_3_list=[df103['digit_3'][n], df103['digit_3'][n], df103['digit_3'][n], df103['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df103_to_insert = pd.DataFrame(data_to_insert)
  df103= df103.append(df103_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df103['digit_3'].value_counts()

303    12035
262     8330
204     8215
272     7530
271     7420
106     6765
162     6720
284     6690
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df104['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=8)
  id_list=['p'+str(df104['AI_id'][n]), 'p'+str(df104['AI_id'][n]), 'p'+str(df104['AI_id'][n]), 'p'+str(df104['AI_id'][n]),'p'+str(df104['AI_id'][n]), 'p'+str(df104['AI_id'][n]), 'p'+str(df104['AI_id'][n]), 'p'+str(df104['AI_id'][n]),'p'+str(df104['AI_id'][n])]
  digit_1_list=[df104['digit_1'][n], df104['digit_1'][n], df104['digit_1'][n], df104['digit_1'][n],df104['digit_1'][n], df104['digit_1'][n], df104['digit_1'][n], df104['digit_1'][n],df104['digit_1'][n]]
  digit_2_list=[df104['digit_2'][n], df104['digit_2'][n], df104['digit_2'][n], df104['digit_2'][n],df104['digit_2'][n], df104['digit_2'][n], df104['digit_2'][n], df104['digit_2'][n],df104['digit_2'][n]]
  digit_3_list=[df104['digit_3'][n], df104['digit_3'][n], df104['digit_3'][n], df104['digit_3'][n],df104['digit_3'][n], df104['digit_3'][n], df104['digit_3'][n], df104['digit_3'][n],df104['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df104_to_insert = pd.DataFrame(data_to_insert)
  df104= df104.append(df104_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df104['digit_3'].value_counts()

241    12790
103    11770
172     9960
102     9850
134     9650
239     9220
151     8650
139     8120
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df105['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=13)
  id_list=['p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]),'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]),'p'+str(df105['AI_id'][n]),'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]), 'p'+str(df105['AI_id'][n]),'p'+str(df105['AI_id'][n])]
  digit_1_list=[df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n],df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n],df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n], df105['digit_1'][n],df105['digit_1'][n]]
  digit_2_list=[df105['digit_2'][n], df105['digit_2'][n], df105['digit_2'][n], df105['digit_2'][n],df105['digit_2'][n], df105['digit_2'][n], df105['digit_2'][n], df105['digit_2'][n],df105['digit_2'][n],df105['digit_2'][n], df105['digit_2'][n], df105['digit_2'][n], df105['digit_2'][n],df105['digit_2'][n]]
  digit_3_list=[df105['digit_3'][n], df105['digit_3'][n], df105['digit_3'][n], df105['digit_3'][n],df105['digit_3'][n], df105['digit_3'][n], df105['digit_3'][n], df105['digit_3'][n],df105['digit_3'][n],df105['digit_3'][n], df105['digit_3'][n], df105['digit_3'][n], df105['digit_3'][n],df105['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df105_to_insert = pd.DataFrame(data_to_insert)
  df105= df105.append(df105_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df105['digit_3'].value_counts()

221    12045
104    11535
233    11370
311    10980
101    10935
264    10770
232    10050
202     9870
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df106['text']:
  result_list=EDA(sentence, alpha_sr=0.0, alpha_ri=0.0, alpha_rs=0.5, p_rd=0.5, num_aug=18)
  id_list=['p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]),'p'+str(df106['AI_id'][n]),'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]),'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]),'p'+str(df106['AI_id'][n]),'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]), 'p'+str(df106['AI_id'][n]),'p'+str(df106['AI_id'][n])]
  digit_1_list=[df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n],df106['digit_1'][n],df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n],df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n],df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n], df106['digit_1'][n],df106['digit_1'][n]]
  digit_2_list=[df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n],df106['digit_2'][n],df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n],df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n],df106['digit_2'][n],df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n], df106['digit_2'][n],df106['digit_2'][n]]
  digit_3_list=[df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n],df106['digit_3'][n],df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n],df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n],df106['digit_3'][n],df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n], df106['digit_3'][n],df106['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df106_to_insert = pd.DataFrame(data_to_insert)
  df106= df106.append(df106_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df106['digit_3'].value_counts()

144    12520
331    12100
285    11600
179    11040
242    10660
231    10460
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df107['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=28)
  id_list=['p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]), 'p'+str(df107['AI_id'][n]),'p'+str(df107['AI_id'][n])]
  digit_1_list=[df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n],df107['digit_1'][n],df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n],df107['digit_1'][n],df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n],df107['digit_1'][n],df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n],df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n],df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n], df107['digit_1'][n],df107['digit_1'][n]]
  digit_2_list=[df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n],df107['digit_2'][n],df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n],df107['digit_2'][n],df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n],df107['digit_2'][n],df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n],df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n],df107['digit_2'][n],df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n], df107['digit_2'][n],df107['digit_2'][n]]
  digit_3_list=[df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n],df107['digit_3'][n],df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n],df107['digit_3'][n],df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n],df107['digit_3'][n],df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n],df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n],df107['digit_3'][n],df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n], df107['digit_3'][n],df107['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df107_to_insert = pd.DataFrame(data_to_insert)
  df107= df107.append(df107_to_insert, ignore_index=True)
  n=n+1

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df108['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=28)
  id_list=['p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]), 'p'+str(df108['AI_id'][n]),'p'+str(df108['AI_id'][n])]
  digit_1_list=[df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n],df108['digit_1'][n],df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n],df108['digit_1'][n],df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n],df108['digit_1'][n],df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n],df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n],df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n], df108['digit_1'][n],df108['digit_1'][n]]
  digit_2_list=[df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n],df108['digit_2'][n],df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n],df108['digit_2'][n],df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n],df108['digit_2'][n],df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n],df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n],df108['digit_2'][n],df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n], df108['digit_2'][n],df108['digit_2'][n]]
  digit_3_list=[df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n],df108['digit_3'][n],df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n],df108['digit_3'][n],df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n],df108['digit_3'][n],df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n],df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n],df108['digit_3'][n],df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n], df108['digit_3'][n],df108['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df108_to_insert = pd.DataFrame(data_to_insert)
  df108= df108.append(df108_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df107['digit_3'].value_counts()

152    15420
131    14220
Name: digit_3, dtype: int64

In [ ]:
df108['digit_3'].value_counts()

283    12360
243    12090
261    11730
289    10620
201    10140
161    10050
203    10020
302     9690
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df109['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=38)
  id_list=['p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]), 'p'+str(df109['AI_id'][n]),'p'+str(df109['AI_id'][n])]
  digit_1_list=[df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n], df109['digit_1'][n],df109['digit_1'][n]]
  digit_2_list=[df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n],df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n], df109['digit_2'][n],df109['digit_2'][n]]
  digit_3_list=[df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n],df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n], df109['digit_3'][n],df109['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df109_to_insert = pd.DataFrame(data_to_insert)
  df109= df109.append(df109_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df109['digit_3'].value_counts()

143    12160
333    11800
108    11400
263    11360
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df110['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=48)
  id_list=['p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]), 'p'+str(df110['AI_id'][n]),'p'+str(df110['AI_id'][n])]
  digit_1_list=[df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n], df110['digit_1'][n],df110['digit_1'][n]]
  digit_2_list=[df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n],df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n], df110['digit_2'][n],df110['digit_2'][n]]
  digit_3_list=[df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n],df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n], df110['digit_3'][n],df110['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df110_to_insert = pd.DataFrame(data_to_insert)
  df110= df110.append(df110_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df110['digit_3'].value_counts()

111    12500
265    11950
133    10850
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df111['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=58)
  id_list=['p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]), 'p'+str(df111['AI_id'][n]),'p'+str(df111['AI_id'][n])]
  digit_1_list=[df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n], df111['digit_1'][n],df111['digit_1'][n]]
  digit_2_list=[df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n],df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n], df111['digit_2'][n],df111['digit_2'][n]]
  digit_3_list=[df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n],df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n], df111['digit_3'][n],df111['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df111_to_insert = pd.DataFrame(data_to_insert)
  df111= df111.append(df111_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df111['digit_3'].value_counts()

334    12720
171    12360
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df112['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=68)
  id_list=['p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]), 'p'+str(df112['AI_id'][n]),'p'+str(df112['AI_id'][n])]
  digit_1_list=[df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n], df112['digit_1'][n],df112['digit_1'][n]]
  digit_2_list=[df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n],df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n], df112['digit_2'][n],df112['digit_2'][n]]
  digit_3_list=[df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n],df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n], df112['digit_3'][n],df112['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df112_to_insert = pd.DataFrame(data_to_insert)
  df112= df112.append(df112_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df112['digit_3'].value_counts()

273    11900
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df113['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=78)
  id_list=['p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]), 'p'+str(df113['AI_id'][n]),'p'+str(df113['AI_id'][n])]
  digit_1_list=[df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n], df113['digit_1'][n],df113['digit_1'][n]]
  digit_2_list=[df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n],df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n], df113['digit_2'][n],df113['digit_2'][n]]
  digit_3_list=[df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n],df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n], df113['digit_3'][n],df113['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df113_to_insert = pd.DataFrame(data_to_insert)
  df113= df113.append(df113_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df113['digit_3'].value_counts()

212    12480
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df114['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=88)
  id_list=['p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]), 'p'+str(df114['AI_id'][n]),'p'+str(df114['AI_id'][n])]
  digit_1_list=[df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n], df114['digit_1'][n],df114['digit_1'][n]]
  digit_2_list=[df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n],df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n], df114['digit_2'][n],df114['digit_2'][n]]
  digit_3_list=[df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n],df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n], df114['digit_3'][n],df114['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df114_to_insert = pd.DataFrame(data_to_insert)
  df114= df114.append(df114_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df114['digit_3'].value_counts()

112    12330
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df115['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=98)
  id_list=['p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]), 'p'+str(df115['AI_id'][n]),'p'+str(df115['AI_id'][n])]
  digit_1_list=[df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n], df115['digit_1'][n],df115['digit_1'][n]]
  digit_2_list=[df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n],df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n], df115['digit_2'][n],df115['digit_2'][n]]
  digit_3_list=[df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n],df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n], df115['digit_3'][n],df115['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df115_to_insert = pd.DataFrame(data_to_insert)
  df115= df115.append(df115_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df115['digit_3'].value_counts()

213    11800
313    10600
319    10100
312     9300
282     8400
332     7600
192     6800
142     6600
Name: digit_3, dtype: int64

In [ ]:
temp_result=[]
n=0
id_list=[]
digit_1_list=[]
digit_2_list=[]
digit_3_list=[]

for sentence in df116['text']:
  result_list=EDA(sentence, alpha_sr=0.1, alpha_ri=0.1, alpha_rs=0.4, p_rd=0.4, num_aug=198)
  id_list=['p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]), 'p'+str(df116['AI_id'][n]),'p'+str(df116['AI_id'][n])]
  digit_1_list=[df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n], df116['digit_1'][n],df116['digit_1'][n]]
  digit_2_list=[df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n],df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n], df116['digit_2'][n],df116['digit_2'][n]]
  digit_3_list=[df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n],df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n], df116['digit_3'][n],df116['digit_3'][n]]
  data_to_insert={'AI_id' : id_list, 'digit_1' : digit_1_list, 'digit_2' : digit_2_list, 'digit_3': digit_3_list, 'text' : result_list}
  df116_to_insert = pd.DataFrame(data_to_insert)
  df116= df116.append(df116_to_insert, ignore_index=True)
  n=n+1

In [ ]:
df116['digit_3'].value_counts()

105    12800
205    11200
274     9400
163     9000
211     7800
182     7600
252     6600
304     4400
301     4000
191     4000
266     1600
120      600
Name: digit_3, dtype: int64

기존 데이터 합치기

In [ ]:
df=pd.concat([df100, df101, df102, df103, df104, df105, df106, df107, df108, df109, df110, df111, df112, df113, df114, df115, df116])
df.reset_index(drop=True)

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000044,C,25,259,금속재료 주문에 의한 절삭가공 공작기계
1,id_0000132,C,25,259,구입한 쇠를 깎고. 연마해서 기계부품 생산
2,id_0000239,C,25,259,철판 프레스가공 모터에 들어가는 코아
3,id_0000252,C,25,259,제조 기계가공 전자부품
4,id_0000372,C,25,259,가공된 금속재료로 절삭가공하여 선반 제조
...,...,...,...,...,...
830866,pid_0992753,C,12,120,"담뱃잎, 담보 잎줄기 슬러리 제조 원료 투입,"
830867,pid_0992753,C,12,120,"담뱃잎, 담보 잎줄기 원료 투입, 슬러리 제조"
830868,pid_0992753,C,12,120,"담뱃잎, 담보 잎줄기 원료 투입, 슬러리 제조"
830869,pid_0992753,C,12,120,"잎줄기 담보 투입, 원료 담뱃잎, 슬러리 제조"


In [ ]:
df['digit_3'].value_counts().sum()

830871

In [ ]:
df

,AI_id,digit_1,digit_2,digit_3,text
0,id_0000044,C,25,259,금속재료 주문에 의한 절삭가공 공작기계
1,id_0000132,C,25,259,구입한 쇠를 깎고. 연마해서 기계부품 생산
2,id_0000239,C,25,259,철판 프레스가공 모터에 들어가는 코아
3,id_0000252,C,25,259,제조 기계가공 전자부품
4,id_0000372,C,25,259,가공된 금속재료로 절삭가공하여 선반 제조
...,...,...,...,...,...
78995,pid_0992753,C,12,120,"담뱃잎, 담보 잎줄기 슬러리 제조 원료 투입,"
78996,pid_0992753,C,12,120,"담뱃잎, 담보 잎줄기 원료 투입, 슬러리 제조"
78997,pid_0992753,C,12,120,"담뱃잎, 담보 잎줄기 원료 투입, 슬러리 제조"
78998,pid_0992753,C,12,120,"잎줄기 담보 투입, 원료 담뱃잎, 슬러리 제조"


In [ ]:
df.to_csv('df_C_inc.csv', index=False, encoding='utf-8-sig')